In [1]:
import os
import re
import pymatgen as mg
import pymatgen.analysis.diffraction as anadi
import pymatgen.analysis.diffraction.xrd as xrd
import numpy as np
import glob
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import time

In [2]:
torch.set_default_dtype(torch.float64)

torch.set_printoptions(precision=8)

patt_xrd = xrd.XRDCalculator('CuKa')

train_path='/home/hjj/Desktop/GANsurface/train/'

test_path='/home/hjj/Desktop/GANsurface/test/'

global sample_num, rmat_num, series_num
sample_num=1 #output of G
rmat_num=28  #row nums of the matrix for the input of CNN 
series_num=3
#input of D
global A
A =12.8282906600000004**2

In [3]:

class DNet(nn.Module):
    def __init__(self):
        super(DNet, self).__init__()
        self.Dlstm=nn.LSTM(
            input_size=series_num,
            hidden_size=32,
            num_layers=1,
            batch_first=True,
        )
        self.out=nn.Sequential(
            nn.Linear(32,10),
            nn.ReLU(),
            nn.Linear(10,1),
            nn.Sigmoid(),
        )
        #nn.Linear(32,1)
        #nn.Relu
        #nn.Linear
        #nn.Sigmoid
        
    def forward(self,x):
        D_out,(h_n,h_c)=self.Dlstm(x,None)
        out = self.out(D_out[:,-1,:]) #(batch,time step,input)   
        return out

In [4]:
def get_slab_energy(folder):
    energy_string=os.popen('grep TOTEN '+folder+'/OUTCAR | tail -1').read().split(' ')[-2]
    energy_slab=round(np.float64(float(energy_string)),5)
    return energy_slab
def get_bluk_energy(folder):
    energy_string=os.popen('grep TOTEN '+folder+'/OUTCAR_bulk | tail -1').read().split(' ')[-2]
    energy_bluk=round(np.float64(float(energy_string)),5)
    return energy_bluk


def get_cpx222_surface_energy(folder):
    slab=get_slab_energy(folder)
    bluk=get_bluk_energy(folder)
    
    global A
    sur_e= (slab-bluk)/(2*A)
    return sur_e

def linear_transform(energy):
    global extend_num, move_num
    energy_transform=(energy-move_num)*extend_num
    return energy_transform
def inverse_transform(energy_transform):
    global extend_num, move_num
    energy=energy_transform/extend_num+move_num
    return energy


In [5]:
global extend_num, move_num

In [6]:
extend_num=1000

In [7]:
move_num=get_cpx222_surface_energy('/home/hjj/Desktop/GANsurface/train/00000')
print(move_num)

0.13767811697355392


In [8]:
base=mg.Structure.from_file(train_path+'00000/POSCAR')
base_xrd =patt_xrd.get_pattern(base)
base_x=[]
base_y=[]
for i in range(len(base_xrd.y)):
    if base_xrd.y[i] >0.0001 and base_xrd.y[i] < 1:
        base_y.append(base_xrd.y[i])
        base_x.append(base_xrd.x[i])
base_x=base_x[:28]
base_y=base_y[:28]

/home/hjj/anaconda3/envs/tensorflow/lib/python3.6/site-packages/pymatgen/io/cif.py:44: UserWarning: Please install optional dependency pybtex if youwant to extract references from CIF files.
  warnings.warn("Please install optional dependency pybtex if you"


In [9]:
def random_xxpsk(file_path):
    folder=np.random.choice(glob.glob(file_path +"*"))
    #pos_name=folder+'/POSCAR'
    #out_name=folder+'/OUTCAR'
    return folder

def tomgStructure(folder):
    POSfile=folder+'/POSCAR'      
    R_mgS=mg.Structure.from_file(POSfile)
    return R_mgS

###
##input_data_to_model
###

def get_xrdmat_slab(mgStructure):
    global rmat_num
    xrd_data4 =patt_xrd.get_pattern(mgStructure)

    i_column = 28
    xxx=[]
    yyy=[]
    mat4=[]
    xrd_i=len(xrd_data4)
    for i in range(xrd_i):
        if xrd_data4.y[i] >0.0001 and xrd_data4.y[i] < 1:
            xxx.append(xrd_data4.x[i])
            yyy.append(xrd_data4.y[i])
    mat4.append(np.asarray(xxx))
    mat4.append(np.asarray(yyy))
    mat4=np.asarray(mat4)
    
    xrd_x=[]
    xrd_y=[]
    xrd_mat4=[]
    xrow=len(mat4[0])
    
    if xrow < i_column:
        for i in mat4[0]:
            xrd_x.append(i)
        for j in mat4[1]:
            xrd_y.append(j)
        for i in range(0,i_column-xrow):
            xrd_x.append(0)
            xrd_y.append(0)
        xrd_x=np.asarray(xrd_x)
        xrd_y=np.asarray(xrd_y)
    if xrow > i_column:
        xrd_x=mat4[0][:i_column]
        xrd_y=mat4[1][:i_column]
    if xrow == i_column:
        xrd_x= mat4[0]
        xrd_y= mat4[1]
        
    #xrd_x=xrd_x-base_x
    xrd_y=1000*(xrd_y-base_y)
    
    xrd_x=np.sin(np.dot(1/180*np.pi,xrd_x))
    xrd_y=(np.arctan(xrd_y))/180*np.pi
    xrd_mat4.append(xrd_x)
    xrd_mat4.append(xrd_y)
    xrd_mat4=np.array(xrd_mat4)
    return xrd_mat4

###
##input_data_as_knowlegde
###
'''
def get_Gibbs(folder):
    energy_string=os.popen('grep TOTEN '+folder+'/OUTCAR | tail -1').read().split(' ')[-2]
    Gibbs=np.float64(float(energy_string))
    Gibbs=round(Gibbs,6)
    return Gibbs
'''
##
###
def get_atoms_num(folder2):
    xxx=tomgStructure(folder2)
    anum=len(xxx.sites)
    return anum


###
##input_data_for_G
###
def GANs_Gmat(Random_Structure):
    global rmat_num
    RS_xrdmat = get_xrdmat_slab(Random_Structure)
    multimat3_RS =  np.zeros((rmat_num,rmat_num),dtype='float32')
    multimat3_RS = np.asarray((np.dot(RS_xrdmat.T, RS_xrdmat)))
    return multimat3_RS

In [10]:
class GNet(nn.Module):
    
    def __init__(self, input_size=(sample_num,28,28)):
        super(GNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(#(3,28,28)
                in_channels=sample_num,
                out_channels=32,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(32,28,28)
            nn.ReLU(),#->(32,28,28)
            nn.MaxPool2d(kernel_size=2),
        )#->(#->(32,14,14))
        self.conv2=nn.Sequential(#->(32,14,14))
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(64,14,14)
            nn.ReLU(),#->(64,14,14)
            nn.MaxPool2d(kernel_size=2),#->(64,7,7)
        )
        self.out=nn.Sequential(
            nn.Linear(64*7*7,128),
            nn.ReLU(),
            nn.Linear(128,sample_num),            
        )
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x) #batch(64,7,7)
        x=x.view(x.size(0),-1) #(batch, 64*7*7)
        output=torch.unsqueeze(self.out(x),dim=0)
        return output


In [11]:
G1=GNet()
D1=DNet()

In [12]:
opt_D1=torch.optim.Adam(D1.parameters(),lr=0.1)
opt_G1=torch.optim.Adam(G1.parameters(),lr=0.1)

In [13]:
train_series=[]
for i in range(series_num):
    path_s=random_xxpsk(train_path)
    ee1=get_cpx222_surface_energy(path_s)
    ee1=linear_transform(ee1)
    train_series.append(ee1)

In [14]:
mat_Gl=[]    
mat_Dl=[]
pre_dd=[]
pre_gg=[]
error_test=[]
error_train=[]


In [15]:
mg.Structure.from_file('/home/hjj/Desktop/GANsurface/train/00002/POSCAR')

Structure Summary
Lattice
    abc : 20.0 12.82829066 12.82829066
 angles : 90.0 90.0 90.0
 volume : 3291.3008251488645
      A : 20.0 0.0 0.0
      B : 0.0 12.82829066 0.0
      C : 0.0 0.0 12.82829066
PeriodicSite: Cs (3.2071, 3.2071, 3.2071) [0.1604, 0.2500, 0.2500]
PeriodicSite: Cs (9.6212, 3.2071, 3.2071) [0.4811, 0.2500, 0.2500]
PeriodicSite: Cs (3.2071, 9.6212, 3.2071) [0.1604, 0.7500, 0.2500]
PeriodicSite: Cs (9.6212, 9.6212, 3.2071) [0.4811, 0.7500, 0.2500]
PeriodicSite: Cs (3.2071, 3.2071, 9.6212) [0.1604, 0.2500, 0.7500]
PeriodicSite: Cs (9.6212, 3.2071, 9.6212) [0.4811, 0.2500, 0.7500]
PeriodicSite: Cs (3.2071, 9.6212, 9.6212) [0.1604, 0.7500, 0.7500]
PeriodicSite: Cs (9.6212, 9.6212, 9.6212) [0.4811, 0.7500, 0.7500]
PeriodicSite: Pb (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Pb (6.4141, 0.0000, 0.0000) [0.3207, 0.0000, 0.0000]
PeriodicSite: Pb (0.0000, 6.4141, 0.0000) [0.0000, 0.5000, 0.0000]
PeriodicSite: Pb (6.4141, 6.4141, 0.0000) [0.3207, 0.5000, 0.

In [16]:
file_path=train_path
tfset=[]
for step in range(1,4):       
 
    sample_path=[]
    for i in range(1,sample_num + 1):
        path_ = random_xxpsk(file_path)
        sample_path.append(path_)
        tfset.append(path_)
    E_Gibbs=0
    for path1_ in sample_path:
        try:
            total_energy=get_cpx222_surface_energy(path1_)
            E_Gibbs=linear_transform(total_energy)
            #print(samp_Gibbs)
        except:
            print(path1_)
         
        train_series.pop(-1)
        train_series.append(E_Gibbs)
        
        
    input_series_D=np.asarray(train_series,dtype=np.float64)       
    input_series_D=Variable(torch.from_numpy(input_series_D[np.newaxis,np.newaxis,:]),requires_grad=True)
    
    prob_Tfactor_mat0=D1(input_series_D)
    pre_dd.append(prob_Tfactor_mat0.data.numpy().mean())
    
    g_in=[]
    for path2_ in sample_path:
        path2_=str(path2_)                
        
        try:
            tomgS=tomgStructure(path2_)
            #print(tomgS)
            gin=GANs_Gmat(tomgS)
            
            #print(gin)
        except:
            pass
        g_in.append(gin)
       
    g_in=np.asarray(g_in)
    g_in=g_in[np.newaxis,:,:,:] 
    g_in=np.asarray(g_in,dtype=np.float64) 
    g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
    Gout=G1(g_in)
    Gout=round(Gout.data.numpy().mean(),6)
    train_series.append(Gout)
    train_series.pop(0)
        
    input_series_D=np.asarray(train_series,dtype=np.float64)       
    input_series_D=Variable(torch.from_numpy(input_series_D[np.newaxis,np.newaxis,:]),requires_grad=True)
    
    
    prob_G1_mat1=D1(input_series_D)
    pre_gg.append(prob_G1_mat1.data.numpy().mean())
    
    D1_loss=-torch.mean(torch.log(prob_Tfactor_mat0)+torch.log(1.-prob_G1_mat1))
    dd=D1_loss.data.numpy().mean()
    mat_Dl.append(dd)
    
    G1_loss=torch.mean(torch.log(1.-prob_G1_mat1))
    gg=G1_loss.data.numpy().mean()
    mat_Gl.append(gg)
    
    if step%3==0:
        opt_D1.zero_grad()
        D1_loss.backward(retain_graph=True)
        opt_D1.step()
        
        opt_G1.zero_grad()
        G1_loss.backward()
        opt_G1.step()
    else:
        opt_D1.zero_grad()
        D1_loss.backward()
        opt_D1.step()
    


    if step%1==0:
        print(step)
        print('error: ',abs(inverse_transform(Gout)-inverse_transform(E_Gibbs)))
        
        print(dd)
        print(gg)
        print(prob_Tfactor_mat0.data.numpy().mean())
        print(prob_G1_mat1.data.numpy().mean())
        
        
        
        
    
    
    if step%3==0:
        print('----------------------------------test-------------------------')
        file_path_test=test_path
        E_Gibbs_1=[]
        E_Gmodel_1=[]
        for step_test in range(10):
            sample_path=[]
            for i in range(1,sample_num + 1):
                path_ = random_xxpsk(file_path_test)
                sample_path.append(path_)
            
            for path1_ in sample_path:
                try:
                    total_energy=get_cpx222_surface_energy(path1_)
                    tt_energy=linear_transform(total_energy)
            #print(samp_Gibbs)
                except:
                    print(path1_)
                    
            tt_energy=inverse_transform(float(tt_energy))
            print('DFT',tt_energy)
            E_Gibbs_1.append(tt_energy)
        
                   

        #print(tfactor.shape)
    
            g_in=[]
            for path2_ in sample_path:
                path2_=str(path2_)                
        
                try:
                    tomgS=tomgStructure(path2_)
            #print(tomgS)
                    gin=GANs_Gmat(tomgS)
                    
            #print(gin)
                except:
                    pass
                g_in.append(gin)
       
            g_in=np.asarray(g_in)
            g_in=g_in[np.newaxis,:,:,:] 
            g_in=np.asarray(g_in,dtype=np.float64) 
            g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
            Gout=G1(g_in)
        
        #print(Gout.shape)
    
            G_data=round(inverse_transform(Gout.data.numpy().mean()),6)
            print('G_predict',G_data)
            E_Gmodel_1.append(G_data)

        
            
            
            print('error',abs(tt_energy-G_data))
            
        print('------------------end-test----------------------------')
        xxx=abs(abs(np.asarray(E_Gibbs_1))-abs(np.asarray(E_Gmodel_1))).mean()
        print(xxx)
        error_test.append(xxx)
        
        X=np.asarray(E_Gibbs_1)
        Y=np.asarray(E_Gmodel_1)

        xbar=X.mean()
        ybar=Y.mean()
        SSR=0
        varX=0
        varY=0
        for i in range(len(X)):
            diffxxbar=X[i]-xbar
            diffyybar=Y[i]-ybar
            SSR+=(diffxxbar*diffyybar)
            varX+=diffxxbar**2
            varY+=diffyybar**2
    
        SST=math.sqrt(varX+varY)
        R2=(SSR/SST)**2
        print("R2:",R2)
        #r2.append(R2)
    else:
        pass
    


1
error:  4.2217416058948354e-05
1.3927538739308805
-0.7787715318105286
0.5411913628227828
0.5410305054522705
2
error:  7.62149999999906e-05
1.4352700865751458
-0.5364981776481704
0.40706927103157936
0.41520748954505715
3
error:  4.2217416058948354e-05
1.3918966670297364
-0.7304271308221303
0.5160923605481456
0.5182968041392688
----------------------------------test-------------------------
/home/hjj/Desktop/GANsurface/test/00047


NameError: name 'tt_energy' is not defined

In [20]:

E_Gibbs_test=[]
E_Gmodel_test=[]
abserrset=[]
MSEset=[]
err0set=[]
testfile=[]
for m1,n1,fname in os.walk(test_path):
    for ieach in n1:
        ieach=test_path+ieach
        testfile.append(ieach)
start=time.time()        
for path_ in testfile:
    try:
        GGG=get_cpx222_surface_energy(path_)
        #GGG=inverse_transform(GGG)
        E_Gibbs_test.append(GGG)
        
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        G_data=Gout.data.numpy().mean()
        G_data=inverse_transform(G_data)
        #G_data=get_energy_per_atom(G_data)
        E_Gmodel_test.append(G_data)
        #print(G_data)
        #print(GGG)
        abserr=abs(G_data-GGG)
        mse=(G_data-GGG)**2
        abserrset.append(abserr)
        MSEset.append(mse)
        err0=abs(abserr/GGG)
        err0set.append(err0)
    except:
        print(path_)
end=time.time()
print(end-start)

1818.6451070308685


In [21]:
print(np.asarray(abserrset).mean())

print(np.asarray(MSEset).mean())

print(np.sqrt(np.asarray(MSEset).mean()))

0.00011977779882471464
2.1182996616444365e-08
0.000145543796214213


In [ ]:
print(abserrset)

In [17]:
E_Gibbs_t=[]
E_Gmodel_t=[]
abs_t_errset=[]
err_t_0set=[]
tMSEset=[]
testfile=[]
for m1,n1,fname in os.walk(train_path):
    for ieach in n1:
        ieach=train_path+ieach
        testfile.append(ieach)

In [ ]:
print(testfile)

In [18]:

#'''        
for path_ in testfile:
    try:
        GGG=get_cpx222_surface_energy(path_)
        
        #GGG=get_energy_per_atom(GGG)
        E_Gibbs_t.append(GGG)
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        G_data=Gout.data.numpy().mean()
        G_data=inverse_transform(G_data)
        #G_data=get_energy_per_atom(G_data)
        E_Gmodel_t.append(G_data)
        #print(G_data)
        #print(GGG)
        abserr=abs(G_data-GGG)
        tmse=(G_data-GGG)**2
        tMSEset.append(tmse)
        abs_t_errset.append(abserr)
        err0=abs(abserr/GGG)
        err_t_0set.append(err0)
    except:
        print(path_)

In [19]:
print(np.asarray(abs_t_errset).mean())

print(np.asarray(tMSEset).mean())

6.909317651984659e-05
5.161632932457264e-09


In [ ]:
print(np.sqrt(np.asarray(tMSEset).mean()))

In [23]:
torch.save(G1.state_dict(),"/home/hjj/Desktop/GANsur2_61_G.pkl") 
torch.save(D1.state_dict(),"/home/hjj/Desktop/GANsur2_61_D.pkl")

In [ ]:
'''
sample_path=[]
sample_path.append('/home/mii/Desktop/cpx222/train/00000')
g_in=[]
for path2_ in sample_path:
    path2_=str(path2_)                
        
    try:
        tomgS=tomgStructure(path2_)
            #print(tomgS)
        gin=GANs_Gmat(tomgS)
                
            #print(gin)
    except:
        pass
    g_in.append(gin)
       
g_in=np.asarray(g_in)
g_in=g_in[np.newaxis,:,:,:] 
g_in=np.asarray(g_in,dtype=np.float64) 
g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
Gout=G1(g_in)
        
        #print(Gout.shape)
    
G_data=round(inverse_transform(Gout.data.numpy().mean()),6)
print('G_predict',G_data)
'''